In [1]:
# Задание 1 Используем файл keywords.csv.
# Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. 
# Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. 
# Если поисковый запрос не содержит названия города, то ставим 'undefined'. 
# Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

# Результат классификации запишите в отдельный столбец region.

In [21]:
geo_data = {
 
    'Центр': ['яндекс', 'вк', 'ярославль'],
 
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
 
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
 
}

In [28]:
import pandas as pd
keywords = pd.read_csv('keywords.csv')
keywords.head()

def geo_keywords(keywords):
    return pd.Series([ find_keyword(keywords['keyword'],geo_data) ])

def find_keyword(keyword,geo_data):
    for key in geo_data:
        if keyword in geo_data[key]:
            return key
    return 'undefined'
     
keywords['region'] = keywords.apply(geo_keywords, axis=1)
keywords.head(10)

,keyword,shows,region
0,вк,64292779,Центр
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined
5,одноклассники моя страница,19976138,undefined
6,майл,15144673,undefined
7,авито,14786593,undefined
8,переводчик,14453469,undefined
9,яндекс,13344734,Центр


In [4]:
# Задание 2 Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
#  - оценка 2 и меньше - низкий рейтинг
#  - оценка 4 и меньше - средний рейтинг
#  - оценка 4.5 и 5 - высокий рейтинг
 
# Результат классификации запишите в столбец class

In [5]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings_avg = ratings.groupby('movieId').mean().reset_index()[['movieId', 'rating']]

def class_function(ratings_avg):       
        if ratings_avg['rating'] <= 2:
            return 'low'

        elif ratings_avg['rating'] <= 4:
            return 'medium'

        elif ratings_avg['rating'] <= 5:
            return 'high'


ratings_avg['class'] = ratings_avg.apply(class_function, axis=1)
ratings_avg.head()

,movieId,rating,class
0,1,3.872470,medium
1,2,3.401869,medium
2,3,3.161017,medium
3,4,2.384615,medium
4,5,3.267857,medium


In [6]:
# Задание 3 Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). 
# Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. 
# Ответ дайте в днях.

In [7]:
import pandas as pd

ratings = pd.read_csv('ml-latest-small/ratings.csv')

r_count = ratings.groupby('userId').count().reset_index()[['userId', 'rating']]

cineman_id = r_count[ r_count['rating'] >= 100 ]

cineman_id_list = cineman_id['userId'].tolist()

cineman_data = ratings[ ratings['userId'].isin(cineman_id_list) ]

cineman_max = cineman_data.groupby('userId').max().reset_index()

cineman_max.columns = ['userId', 'movieId', 'rating', 'max_timestamp']

cineman_min = cineman_data.groupby('userId').min().reset_index()

cineman_min.columns = ['userId', 'movieId', 'rating', 'min_timestamp']

cineman_min_max = cineman_max.merge(cineman_min, on='userId', how='left')

cineman_min_max.drop(cineman_min_max.columns[[1,2,4,5]], axis=1, inplace=True)

cineman_min_max['lifetime'] = cineman_min_max.apply(lambda x: ( (x['max_timestamp'] - x['min_timestamp']) / 86400 ), axis=1)

lifetime_days = cineman_min_max['lifetime'].mean()

lifetime_days

455.22857137198986

In [8]:
# Задание 4 Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение,
# используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска 
# фильма его средний рейтинг становится ниже.

# При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:
 
# 1. В переменную years запишите список из всех годов с 1950 по 2010.
 
# 2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. 
#    Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
#  - для каждой строки пройдите по всем годам списка years
#  - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
#  - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
 
# 3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'
 
# 4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по 
# убыванию рейтинга

In [49]:
import pandas as pd
import numpy as np

movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')

years = []
for i in range(1950, 2011):
    years.append(str(i))

def production_year(movies):
    return pd.Series( [ year_search(movies['title'], years) ] )

def year_search(title, years):
    for i in years:
        if i in title:
            return i
    return '1990'
    
movies['year'] = movies.apply(production_year, axis=1)

movies_year_rating = movies.merge(ratings, on='movieId', how='left')
movies_year_rating.drop(movies_year_rating.columns[[2,4,6]], axis=1, inplace=True)

movies_year_mean = movies_year_rating.groupby('year').mean().reset_index().head()
movies_year_mean.sort_values('rating', ascending=False).head()

,year,movieId,rating
2,1952,4462.532468,4.000000
4,1954,2867.661850,3.994220
1,1951,2605.588477,3.983539
0,1950,3503.427966,3.915254
3,1953,4047.401826,3.754587
